# Fetch and parse ACS benchmark results under a given directory
Each ACS benchmark run outputs a json file. This script collects all such files under a given root directory, parses them, and aggregates them into a more easily digestable pandas DataFrame.

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

Set the local path to the root results directory:

In [2]:
RESULTS_ROOT_DIR = Path("/fast/groups/sf") / "folktexts-results" / "2024-06-30"

Set the local path to the root data directory (needed only to train baseline ML methods):

In [3]:
DATA_DIR = Path("/fast/groups/sf") / "data"

Important results columns:

In [4]:
model_col = "config_model_name"
task_col = "config_task_name"

feature_subset_col = "config_feature_subset"
population_subset_col = "config_population_filter"
predictions_path_col = "predictions_path"

Helper function to parse each dictionary containing benchmark results:

In [5]:
from utils import (
    num_features_helper,
    parse_model_name,
    get_non_instruction_tuned_name,
    prettify_model_name,
)

def parse_results_dict(dct) -> dict:
    """Parses results dict and brings all information to the top-level."""

    # Make a copy so we don't modify the input object
    dct = dct.copy()

    # Discard plots' paths
    dct.pop("plots", None)

    # Bring configs to top-level
    config = dct.pop("config", {})
    for key, val in config.items():
        dct[f"config_{key}"] = val

    # Parse model name
    dct[model_col] = parse_model_name(dct[model_col])
    dct["base_name"] = get_non_instruction_tuned_name(dct[model_col])
    dct["name"] = prettify_model_name(dct[model_col])

    # Is instruction-tuned model?
    dct["is_inst"] = dct["base_name"] != dct[model_col]

    # Log number of features
    dct["num_features"] = num_features_helper(dct[feature_subset_col], max_features_return=-1)
    dct["uses_all_features"] = (dct[feature_subset_col] is None) or (dct["num_features"] == -1)

    if dct[feature_subset_col] is None:
        dct[feature_subset_col] = "full"

    # Assert all results are at the top-level
    assert not any(isinstance(val, dict) for val in dct.values())
    return dct


Iteratively search the root directory for results files matching the given regex:

In [6]:
from utils import find_files, load_json

# Results file name pattern
pattern = r'^results.bench-(?P<hash>\d+)[.]json$'

# Find results files and aggregate
results = {}
for file_path in tqdm(find_files(RESULTS_ROOT_DIR, pattern)):
    results[Path(file_path).parent.name] = parse_results_dict(load_json(file_path))

if len(results) == 0:
    raise RuntimeError(f"Couldn't find any results at {RESULTS_ROOT_DIR}")
else:
    print(f"Found {len(results)} benchmark results.")

0it [00:00, ?it/s]

Found 94 benchmark results.


Aggregate results into a single DataFrame, and generate a unique identifier for each row:

In [7]:
df = pd.DataFrame(list(results.values()))

def row_id(row) -> str:
    """Unique row identifier."""
    return f"{row[model_col]}__{row[task_col]}__{row['num_features']}"

df["id"] = df.apply(row_id, axis=1)
df = df.set_index("id", drop=True, verify_integrity=True)

print(f"{df.shape=}")
df.sample(3)

df.shape=(94, 58)


,accuracy,accuracy_diff,accuracy_ratio,balanced_accuracy,balanced_accuracy_diff,balanced_accuracy_ratio,brier_score_loss,ece,ece_quantile,equalized_odds_diff,...,config_population_filter,config_reuse_few_shot_examples,config_seed,config_task_hash,config_task_name,base_name,name,is_inst,num_features,uses_all_features
id,,,,,,,,,,,,,,,,,,,,,
gemma-2-9b__ACSIncome__-1,0.402944,0.220766,0.514724,0.527615,0.065248,0.885922,0.246766,0.256795,0.254430,0.134841,...,None,False,42,2612382143,ACSIncome,gemma-2-9b,Gemma 2 9B,False,-1,True
Mistral-7B-v0.1__ACSEmployment__-1,0.453446,0.262042,0.473408,0.500000,0.000000,1.000000,0.300982,0.262849,0.273442,0.000000,...,None,False,42,1212564561,ACSEmployment,Mistral-7B-v0.1,Mistral 7B,False,-1,True
gemma-2b__ACSTravelTime__-1,0.438448,0.401434,0.210672,0.499891,0.001378,0.997249,0.255249,0.091138,0.091138,0.002320,...,None,False,42,233993660,ACSTravelTime,gemma-2b,Gemma 2B,False,-1,True


Drop potential duplicates:

In [8]:
parsed_df = df.drop_duplicates(subset=["name", "is_inst", "num_features", task_col])
if len(parsed_df) != len(df):
    print(f"Found {len(df) - len(parsed_df)} duplicates! dropping rows...")
    df = parsed_df

Load scores DFs and analyze score distribution:

In [9]:
def load_model_scores_df(df_row: pd.Series) -> pd.DataFrame:
    """Loads csv containing model scores corresponding to the given DF row."""
    if predictions_path_col in df_row and not pd.isna(df_row[predictions_path_col]):
        return pd.read_csv(df_row[predictions_path_col], index_col=0)
    return None

In [10]:
from folktexts.evaluation import compute_best_threshold
from sklearn import metrics
from scipy import stats

# Number of samples used to fit the one-parameter binarization threshold!
N = 100

fit_thresh_col = f"fit_thresh_on_{N}"
fit_acc_col = f"fit_thresh_accuracy"

optimal_thres_col = "optimal_thresh"
optimal_acc_col = "optimal_thresh_accuracy"

score_stdev_col = "score_stdev"
score_mean_col = "score_mean"

scores_stats = dict()
for row_id, row in tqdm(df.iterrows(), total=len(df)):

    # Load model scores
    scores_df = load_model_scores_df(row)
    if scores_df is None:
        logging.error(f"Couldn't find scores for {row_id}")

    # Extract scores and labels
    risk_scores = scores_df["risk_score"].to_numpy()
    labels = scores_df["label"].to_numpy()

    # Sample N rows to fit threshold
    scores_df_sample = scores_df.sample(n=N, random_state=42)

    # Compute optimal threshold on each data sample
    fit_thr = compute_best_threshold(y_true=scores_df_sample["label"], y_pred_scores=scores_df_sample["risk_score"])
    opt_thr = compute_best_threshold(y_true=labels, y_pred_scores=risk_scores)

    # Evaluate accuracy
    fit_acc = metrics.accuracy_score(labels, (risk_scores >= fit_thr).astype(int))
    opt_acc = metrics.accuracy_score(labels, (risk_scores >= opt_thr).astype(int))

    # Save results
    scores_stats[row_id] = {
        fit_thresh_col: fit_thr,
        fit_acc_col: fit_acc,
        optimal_thres_col: opt_thr,
        optimal_acc_col: opt_acc,
        score_stdev_col: np.std(risk_scores),
        score_mean_col: np.mean(risk_scores),
    }

  0%|          | 0/94 [00:00<?, ?it/s]

Update results DF with scores statistics:

In [11]:
scores_stats_df = pd.DataFrame(scores_stats.values(), index=list(scores_stats.keys()))

results_df = pd.concat((df, scores_stats_df), axis="columns")
results_df.sample(2)

,accuracy,accuracy_diff,accuracy_ratio,balanced_accuracy,balanced_accuracy_diff,balanced_accuracy_ratio,brier_score_loss,ece,ece_quantile,equalized_odds_diff,...,name,is_inst,num_features,uses_all_features,fit_thresh_on_100,fit_thresh_accuracy,optimal_thresh,optimal_thresh_accuracy,score_stdev,score_mean
gemma-2-27b__ACSMobility__-1,0.607096,0.140662,0.805237,0.497202,0.046088,0.912574,0.274996,0.22845,0.227959,0.091793,...,Gemma 2 27B,False,-1,True,0.651423,0.536251,0.060086,0.731520,0.270062,0.664836
Meta-Llama-3-70B-Instruct__ACSTravelTime__-1,0.595602,0.256723,0.687468,0.540876,0.122157,0.790165,0.240557,0.14848,0.148809,0.198067,...,Llama 3 70B (it),True,-1,True,0.245070,0.614189,0.269003,0.638741,0.124155,0.289832


Finally, save results DF to the results root directory:

In [12]:
from datetime import datetime
def get_current_timestamp() -> str:
    """Return a timestamp representing the current time up to the second."""
    return datetime.now().strftime("%Y.%m.%d-%H.%M.%S")

df.to_csv(Path(RESULTS_ROOT_DIR) / f"aggregated_results.{get_current_timestamp()}.csv")

---